In [6]:
## parser.py
import requests
from bs4 import BeautifulSoup
import json
import os
from os import getcwd


## python파일의 위치
BASE_DIR = os.path.dirname(getcwd())


## HTTP GET Request
req = requests.get('https://beomi.github.io/beomi.github.io_old/')
## HTML 소스 가져오기
html = req.text
## BeautifulSoup으로 html소스를 python객체로 변환하기
## 첫 인자는 html소스코드, 두 번째 인자는 어떤 parser를 이용할지 명시.
## 이 글에서는 Python 내장 html.parser를 이용했다.
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
## CSS Selector를 통해 html요소들을 찾아낸다.
my_titles = soup.select(
    'h3 > a'
    )
## my_titles는 list 객체
for title in my_titles:
    ## Tag안의 텍스트
    print(title.text)
    ## Tag의 속성을 가져오기(ex: href속성)
    print(title.get('href'))
    
data = {}

for title in my_titles:
    data[title.text] = title.get('href')

with open(os.path.join(BASE_DIR, 'result.json'), 'w+') as json_file:
    json.dump(data, json_file)
    

In [ ]:
from bs4 import BeautifulSoup
from mitmproxy import ctx

# load in the javascript to inject
with open('content.js', 'r') as f:
    content_js = f.read()

def response(flow):
    # only process 200 responses of html content
    if flow.response.headers['Content-Type'] != 'text/html':
        return
    if not flow.response.status_code == 200:
        return

    # inject the script tag
    html = BeautifulSoup(flow.response.text, 'lxml')
    container = html.head or html.body
    if container:
        script = html.new_tag('script', type='text/javascript')
        script.string = content_js
        container.insert(0, script)
        flow.response.text = str(html)

        ctx.log.info('Successfully injected the content.js script.')

In [1]:

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

options = webdriver.ChromeOptions()
#URL = 'http://localhost:4444/wd/hub'
options.add_argument('--headless')
options.add_argument("--no-sandbox")
#options.add_argument("--disable-gpu")
#options.add_argument("--remote-debugin-port=9222")
options.add_argument("proxy-server=localhost:8080")

options.add_argument("lang=es") # 스페인어!

options.add_argument('window-size=1920x1080')

# UserAgent값을 바꿔줍시다!
options.add_argument("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_4) AppleWebKit/600.7.12 (KHTML, like Gecko) Version/8.0.7 Safari/600.7.12")


driver = webdriver.Chrome('chromedriver',chrome_options=options)

driver.get('about:blank')
driver.execute_script("Object.defineProperty(navigator, 'plugins', {get: function() {return[1, 2, 3, 4, 5];},});")
# lanuages 속성을 업데이트해주기
driver.execute_script("Object.defineProperty(navigator, 'languages', {get: function() {return ['es', 'es']}})")
driver.execute_script("const getParameter = WebGLRenderingContext.getParameter;WebGLRenderingContext.prototype.getParameter = function(parameter) {if (parameter === 37445) {return 'NVIDIA Corporation'} if (parameter === 37446) {return 'NVIDIA GeForce GTX 980 Ti OpenGL Engine';}return getParameter(parameter);};")


In [2]:
TEST_URL = 'https://intoli.com/blog/making-chrome-headless-undetectable/chrome-headless-test.html'

driver.get(TEST_URL)
print(driver.page_source)

user_agent = driver.find_element_by_css_selector('#user-agent')['text']
plugins_length = driver.find_element_by_css_selector('#plugins-length').text
languages = driver.find_element_by_css_selector('#languages').text
webgl_vendor = driver.find_element_by_css_selector('#webgl-vendor').text
webgl_renderer = driver.find_element_by_css_selector('#webgl-renderer').text

print('User-Agent: ', user_agent)
print('Plugin length: ', plugins_length)
print('languages: ', languages)
print('WebGL Vendor: ', webgl_vendor)
print('WebGL Renderer: ', webgl_renderer)
driver.quit()

<html xmlns="http://www.w3.org/1999/xhtml"><head></head><body></body></html>


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#user-agent"}
  (Session info: headless chrome=67.0.3396.79)
  (Driver info: chromedriver=2.40.565383 (76257d1ab79276b2d53ee976b2c3e3b9f335cde7),platform=Linux 4.9.87-linuxkit-aufs x86_64)


In [5]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities


TEST_URL = 'https://intoli.com/blog/making-chrome-headless-undetectable/chrome-headless-test.html'

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
options.add_argument("lang=ko_KR") # 한국어!
driver = webdriver.Chrome(chrome_options=options)

driver.get(TEST_URL)
driver.execute_script("Object.defineProperty(navigator, 'plugins', {get: function() {return[1, 2, 3, 4, 5]}})")
driver.execute_script("Object.defineProperty(navigator, 'languages', {get: function() {return ['ko-KR', 'ko']}})")
driver.execute_script("const getParameter = WebGLRenderingContext.getParameter;WebGLRenderingContext.prototype.getParameter = function(parameter) {if (parameter === 37445) {return 'NVIDIA Corporation'} if (parameter === 37446) {return 'NVIDIA GeForce GTX 980 Ti OpenGL Engine';}return getParameter(parameter);};")

user_agent = driver.find_element_by_css_selector('#user-agent').text
plugins_length = driver.find_element_by_css_selector('#plugins-length').text
languages = driver.find_element_by_css_selector('#languages').text
webgl_vendor = driver.find_element_by_css_selector('#webgl-vendor').text
webgl_renderer = driver.find_element_by_css_selector('#webgl-renderer').text

print('User-Agent: ', user_agent)
print('Plugin length: ', plugins_length)
print('languages: ', languages)
print('WebGL Vendor: ', webgl_vendor)
print('WebGL Renderer: ', webgl_renderer)

driver.quit()


ConnectionResetError: [Errno 104] Connection reset by peer

In [34]:
from bs4 import BeautifulSoup

driver.implicitly_wait(3)
driver.get('https://www.idealista.com/login')
## 아이디/비밀번호를 입력해준다.
driver.find_element_by_name('email').send_keys('pinguino.lim@gmail.com')
driver.find_element_by_name('pwd').send_keys('dpsflRp2013')
## 로그인 버튼을 눌러주자.
driver.find_element_by_xpath('//*[@id="doLogin"]').click()

#driver.get('https://www.idealista.com/alquiler-viviendas/madrid-madrid/') ## Naver 페이 들어가기
driver.get_screenshot_as_file('idealist_main_headless.png')

driver.quit()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"name","selector":"email"}
  (Session info: headless chrome=67.0.3396.79)
  (Driver info: chromedriver=2.40.565383 (76257d1ab79276b2d53ee976b2c3e3b9f335cde7),platform=Linux 4.9.87-linuxkit-aufs x86_64)


In [ ]:
from bs4 import BeautifulSoup

driver.implicitly_wait(10)
driver.get('https://www.idealista.com/login')
## 아이디/비밀번호를 입력해준다.
driver.find_element_by_name('email').send_keys('pinguino.lim@gmail.com')
driver.find_element_by_name('pwd').send_keys('dpsflRp2013')
## 로그인 버튼을 눌러주자.
driver.find_element_by_xpath('//*[@id="doLogin"]').click()

driver.get('https://www.idealista.com/alquiler-viviendas/madrid-madrid/') ## Naver 페이 들어가기
html = driver.page_source ## 페이지의 elements모두 가져오기
soup = BeautifulSoup(html, 'html.parser') ## BeautifulSoup사용하기
notices = soup.select('main-content > div.items-container > article > div > div > div.item-info-container > a')

for n in notices:
    print(n.text.strip())

In [17]:
from bs4 import BeautifulSoup

driver.implicitly_wait(3)
driver.get('https://www.idealista.com/login')
## 아이디/비밀번호를 입력해준다.
driver.find_element_by_name('email')

<selenium.webdriver.remote.webelement.WebElement (session="0f4be230b64a2946026b99b0e6698fdd", element="0.13181746995817267-1")>